In [1]:
import torch, torch.nn.functional as F, torch.nn as nn
from model import custom_res18
from dataset import full_forget_retain_loader_train, full_forget_retain_loader_test
from train_test_acc import train_model, model_test
from mia_unlearning_score import UnLearningScore, get_membership_attack_prob, actv_dist
from seed import set_seed
import gc

In [2]:
set_seed(42)
BATCH = 512
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
retrain = custom_res18(10).to(DEVICE)
retrain.load_state_dict(torch.load('./retrain_tr_model.pth', map_location='cpu'))
retrain.eval()
full_tr, forget_tr, retain_tr = full_forget_retain_loader_train(1, batch_size=BATCH)
full_te, forget_te, retain_te = full_forget_retain_loader_test(1, batch_size=BATCH)

print('\n=== Accuracy (추가: Retrain baseline) === retrain model ')
print('Retrain/Retain_train')
model_test(retrain, retain_tr.dataset, 'Retrain/Retain_train', batch_size=256, device=DEVICE)
print('Retrain/Forget_train')
model_test(retrain, forget_tr.dataset, 'Retrain/Forget_train', batch_size=256, device=DEVICE)
print('Retrain/Retain-test')
model_test(retrain, retain_te.dataset, 'Retrain/Retain-test', batch_size=256, device=DEVICE)
print('Retrain/Forget-test')
model_test(retrain, forget_te.dataset, 'Retrain/Forget-test', batch_size=256, device=DEVICE)





print('\n[MIA]')
forget_eval_test  = torch.utils.data.DataLoader(forget_te.dataset,  batch_size=128, shuffle=False)
retain_eval_train = torch.utils.data.DataLoader(retain_tr.dataset,  batch_size=128, shuffle=False)
retain_eval_test  = torch.utils.data.DataLoader(retain_te.dataset,  batch_size=128, shuffle=False)
mia_p = get_membership_attack_prob(retain_eval_train, forget_eval_test, retain_eval_test, retrain, DEVICE)
print(f'  MIA success prob on Forget (test) : {mia_p:.4f}')

/root/anaconda3/envs/khw/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/anaconda3/envs/khw/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Files already downloaded and verified
Files already downloaded and verified

=== Accuracy (추가: Retrain baseline) === retrain model 
Retrain/Retain_train
[Test:Retrain/Retain_train] Acc 98.16%
Retrain/Forget_train
[Test:Retrain/Forget_train] Acc 0.00%
Retrain/Retain-test
[Test:Retrain/Retain-test] Acc 79.10%
Retrain/Forget-test
[Test:Retrain/Forget-test] Acc 0.00%

[MIA]
  MIA success prob on Forget (test) : 0.8073


In [3]:
set_seed(42)
BATCH = 512
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
finetune = custom_res18(10).to(DEVICE)
finetune.load_state_dict(torch.load('./full_tr_model.pth', map_location='cpu'))
finetune.eval()


print('\n=== Accuracy (추가: finetune baseline) === finetunes model ')
print('finetune/Retain_train')
model_test(finetune, retain_tr.dataset, 'finetune/Retain_train', batch_size=256, device=DEVICE)
print('finetune/Forget_train')
model_test(finetune, forget_tr.dataset, 'finetune/Forget_train', batch_size=256, device=DEVICE)
print('finetune/Retain-test')
model_test(retrain, retain_te.dataset, 'finetune/Retain-test', batch_size=256, device=DEVICE)
print('finetune/Forget-test')
model_test(finetune, forget_te.dataset, 'finetune/Forget-test', batch_size=256, device=DEVICE)





print('\n[MIA]')
forget_eval_test  = torch.utils.data.DataLoader(forget_te.dataset,  batch_size=128, shuffle=False)
retain_eval_train = torch.utils.data.DataLoader(retain_tr.dataset,  batch_size=128, shuffle=False)
retain_eval_test  = torch.utils.data.DataLoader(retain_te.dataset,  batch_size=128, shuffle=False)
mia_p = get_membership_attack_prob(retain_eval_train, forget_eval_test, retain_eval_test, finetune, DEVICE)
print(f'  MIA success prob on Forget (test) : {mia_p:.4f}')

/root/anaconda3/envs/khw/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/anaconda3/envs/khw/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



=== Accuracy (추가: finetune baseline) === finetunes model 
finetune/Retain_train
[Test:finetune/Retain_train] Acc 94.75%
finetune/Forget_train
[Test:finetune/Forget_train] Acc 99.88%
finetune/Retain-test
[Test:finetune/Retain-test] Acc 79.10%
finetune/Forget-test
[Test:finetune/Forget-test] Acc 95.30%

[MIA]
  MIA success prob on Forget (test) : 0.8337
